In [ ]:
include("./BBTNet/datasets/WIDERFACE.jl")
include("configs.jl")

train_data = WIDER_Data(wf_path * "train/")
print("WIDER Data is read. Total images: ", train_data.num_files, " & Total Faces: ", train_data.num_faces, '\n')

In [ ]:
include("./BBTNet/datasets/WIDERFACE.jl")

(imgs, boxes), state = iterate(train_data)
print("Got the first batch: ", size(imgs))

In [ ]:
include("./BBTNet/utils/ImageReader.jl")

train_reader = Image_Reader(true)
test_reader = Image_Reader(false)

filename = "0--Parade/0_Parade_marchingband_1_799.jpg"
img, bboxes = read_img(train_reader, root_dir * "train/images/" * filename, train_data.bboxes[filename], 640)
print("Image file is read: ", filename, '\n')

In [ ]:
include("./BBTNet/utils/draw.jl")

draw_boxes_and_landmarks(img, bboxes)

## Running The Model

In [ ]:
include("./BBTNet/models/retinaface.jl")

# atype = KnetArray{Float32}
atype = Array{Float32}

x = convert(atype, rand(640, 640, 3, 2))
model = RetinaFace(dtype=atype)

c, b, l = model(x, train=false)
print(size(c), " & ", size(b), " & ", size(l), " \n")

In [ ]:
include("BBTNet/models/retinaface.jl")
include("BBTNet/datasets/WIDERFACE.jl")
include("./BBTNet/utils/draw.jl")
include("./BBTNet/utils/box_processes.jl")

atype = KnetArray{Float32}
# atype = Array{Float32}
images_folder_dir = "../Datasets/WIDERFACE/WIDER_"

val_data = WIDER_Data(images_folder_dir * "val/", train=false, batch_size=2)
(imgs, boxes), state = iterate(val_data)
print("Got the first batch: ", size(imgs), "\n")

model = RetinaFace(dtype=atype)
imgs_permuted = convert(atype, permutedims(imgs, (3,2,1,4)))
c, b, l = model(imgs_permuted, train=false)

combined = permutedims(cat(b[1], l[1], dims=2), (2, 1))

draw_boxes_and_landmarks(imgs[:,:,:,1], combined)

In [ ]:
using JLD2
using FileIO

@save "retinaface_rand.jld2" model
model_dict = load("retinaface_rand.jld2")
model2 = model_dict["model"]
print("Model is loaded from a file!\n")

In [ ]:
using Images

include("./BBTNet/utils/draw.jl")

data = reshape([449 330 122 149 488.906 373.643 542.089 376.442 515.031 412.83 485.174 425.893 538.357 431.491], (14, 1))
path = wf_path * "train/images/0--Parade/0_Parade_marchingband_1_849.jpg"

img = channelview(load(path))
draw_boxes_and_landmarks(img, data)

In [ ]:
include("./BBTNet/utils/ImageReader.jl")

rotated = reverse(img, dims=3)
# colorview(RGB, float.(rotated))
size(img)

In [ ]:
using Random, Knet

In [ ]:
include("BBTNet/models/retinaface.jl")
include("BBTNet/datasets/WIDERFACE.jl")
include("configs.jl")

Random.seed!(42)

data = WIDER_Data(wf_path * "train/", wf_labels_path * "train/", train=true, batch_size=1, dtype=atype)

model = RetinaFace(dtype=atype)
train_model(model, data)

In [2]:
using AutoGrad

@doc @primitive1

```
@primitive  fx g1 g2...
```

Define a new primitive operation for AutoGrad and (optionally) specify its gradients. Non-differentiable functions such as `sign`, and non-numeric functions such as `size` should be defined using the @zerograd macro instead.

# Examples

```
@primitive sin(x::Number)
@primitive hypot(x1,x2),dy,y

@primitive sin(x::Number),dy  (dy.*cos(x))
@primitive hypot(x1,x2),dy,y  (dy.*x1./y)  (dy.*x2./y)
```

The first example shows that `fx` is a typed method declaration.  Julia supports multiple dispatch, i.e. a single function can have multiple methods with different arg types. AutoGrad takes advantage of this and supports multiple dispatch for primitives and gradients.

The second example specifies variable names for the output gradient `dy` and the output `y` after the method declaration which can be used in gradient expressions.  Untyped, ellipsis and keyword arguments are ok as in `f(a::Int,b,c...;d=1)`.  Parametric methods such as `f(x::T) where {T<:Number}` cannot be used.

The method declaration can optionally be followed by gradient expressions.  The third and fourth examples show how gradients can be specified.  Note that the parameters, the return variable and the output gradient of the original function can be used in the gradient expressions.

# Under the hood

The @primitive macro turns the first example into:

```
sin(x::Value{T}) where {T<:Number} = forw(sin, x)
```

This will cause calls to `sin` with a boxed argument (`Value{T<:Number}`) to be recorded. The recorded operations are used by AutoGrad to construct a dynamic computational graph. With multiple arguments things are a bit more complicated.  Here is what happens with the second example:

```
hypot(x1::Value{S}, x2::Value{T}) where {S,T} = forw(hypot, x1, x2)
hypot(x1::S, x2::Value{T})        where {S,T} = forw(hypot, x1, x2)
hypot(x1::Value{S}, x2::T)        where {S,T} = forw(hypot, x1, x2)
```

We want the forw method to be called if any one of the arguments is a boxed `Value`.  There is no easy way to specify this in Julia, so the macro generates all 2^N-1 boxed/unboxed argument combinations.

In AutoGrad, gradients are defined using gradient methods that have the following pattern:

```
back(f,Arg{i},dy,y,x...) => dx[i]
```

For the third example here is the generated gradient method:

```
back(::typeof(sin), ::Type{Arg{1}}, dy, y, x::Value{T}) where {T<:Number} = dy .* cos(x)
```

For the last example a different gradient method is generated for each argument:

```
back(::typeof(hypot), ::Type{Arg{1}}, dy, y, x1::Value{S}, x2::Value{T}) where {S,T} = (dy .* x1) ./ y
back(::typeof(hypot), ::Type{Arg{2}}, dy, y, x1::Value{S}, x2::Value{T}) where {S,T} = (dy .* x2) ./ y
```

In fact @primitive generates four more definitions for the other boxed/unboxed argument combinations.

# Broadcasting

Broadcasting is handled by extra `forw` and `back` methods. `@primitive` defines the following  so that broadcasting of a primitive function with a boxed value triggers `forw` and `back`.

```
broadcasted(::typeof(sin), x::Value{T}) where {T<:Number} = forw(broadcasted,sin,x)
back(::typeof(broadcasted), ::Type{Arg{2}}, dy, y, ::typeof(sin), x::Value{T}) where {T<:Number} = dy .* cos(x)
```

If you do not want the broadcasting methods, you can use the `@primitive1` macro. If you only want the broadcasting methods use `@primitive2`. As a motivating example, here is how `*` is defined for non-scalars:

```
@primitive1 *(x1,x2),dy  (dy*x2')  (x1'*dy)
@primitive2 *(x1,x2),dy  unbroadcast(x1,dy.*x2)  unbroadcast(x2,x1.*dy)
```

Regular `*` is matrix multiplication, broadcasted `*` is elementwise multiplication and the two have different gradients as defined above. `unbroadcast(a,b)` reduces `b` to the same shape as `a` by performing the necessary summations.


In [3]:
using Pycall

@pyimport numpy 
@pyimport torch

LoadError: ArgumentError: Package Pycall not found in current path:
- Run `import Pkg; Pkg.add("Pycall")` to install the Pycall package.


In [ ]:
import Pkg; Pkg.add("Pycall")

In [10]:
temp = randn(3, 5)
temp

3×5 Array{Float64,2}:
 0.463706  -1.29645  -1.42806    -0.183817  0.320075
 0.310392   0.61776   0.212497    0.661844  0.632299
 0.102909   1.31899  -0.0961019  -0.413573  0.0163056

In [16]:
idx = findall(temp .> 0.5)
idx

4-element Array{CartesianIndex{2},1}:
 CartesianIndex(2, 2)
 CartesianIndex(3, 2)
 CartesianIndex(2, 4)
 CartesianIndex(2, 5)

In [14]:
temp[!idx]

LoadError: MethodError: no method matching !(::Array{CartesianIndex{2},1})
Closest candidates are:
  !(!Matched::Missing) at missing.jl:100
  !(!Matched::Bool) at bool.jl:33
  !(!Matched::Function) at operators.jl:896

In [10]:
include("BBTNet/datasets/WIDERFACE.jl")
include("configs.jl")

import Random
Random.seed!(42)
data = WIDER_Data(wf_path * "train/", wf_labels_path * "train/", train=true, batch_size=batch_size, dtype=atype)

WIDER_Data("/datasets/widerface/WIDER_train/", Dict{Any,Any}("25--Soldier_Patrol/25_Soldier_Patrol_Soldier_Patrol_25_331.jpg" => [380.0; 96.0; … ; 191.0; 1.0],"14--Traffic/14_Traffic_Traffic_14_811.jpg" => [558.0; 108.0; … ; -1.0; -1.0],"3--Riot/3_Riot_Riot_3_177.jpg" => [277.0 224.0 … 582.0 187.0; 122.0 33.0 … 44.0 46.0; … ; 173.938 -1.0 … -1.0 -1.0; 1.0 -1.0 … -1.0 -1.0],"35--Basketball/35_Basketball_playingbasketball_35_69.jpg" => [620.0; 52.0; … ; 161.076; 1.0],"0--Parade/0_Parade_Parade_0_142.jpg" => [808.0 928.0 … 392.0 237.0; 236.0 219.0 … 51.0 52.0; … ; 277.625 258.34 … -1.0 -1.0; 1.0 1.0 … -1.0 -1.0],"24--Soldier_Firing/24_Soldier_Firing_Soldier_Firing_24_972.jpg" => [378.0; 137.0; … ; 264.067; 1.0],"41--Swimming/41_Swimming_Swimmer_41_150.jpg" => [670.0; 208.0; … ; 286.589; 1.0],"12--Group/12_Group_Large_Group_12_Group_Large_Group_12_516.jpg" => [417.0 498.0; 429.0 366.0; … ; 514.754 468.46; 1.0 1.0],"12--Group/12_Group_Team_Organized_Group_12_Group_Team_Organized_Group_12_43

In [11]:
include("BBTNet/models/retinaface.jl")

model = RetinaFace(dtype=atype)
train_model(model, data)


============================== TRAINING PROCESS ==============================

(640, 640, 3, 16)(16,)

Stacktrace:
 [1] alloc at /kuacc/users/baristopal20/.julia/packages/CUDA/YeS8q/src/pool.jl:298 [inlined]
 [2] CUDA.CuArray{UInt8,1}(::UndefInitializer, ::Tuple{Int64}) at /kuacc/users/baristopal20/.julia/packages/CUDA/YeS8q/src/array.jl:20
 [3] CuArray at /kuacc/users/baristopal20/.julia/packages/CUDA/YeS8q/src/array.jl:76 [inlined]
 [4] CuArray at /kuacc/users/baristopal20/.julia/packages/CUDA/YeS8q/src/array.jl:77 [inlined]
 [5] KnetPtrCu(::Int64) at /kuacc/users/baristopal20/.julia/packages/Knet/C0PoK/src/knetarrays/kptr.jl:229
 [6] Knet.KnetArrays.KnetPtr(::Int64) at /kuacc/users/baristopal20/.julia/packages/Knet/C0PoK/src/knetarrays/kptr.jl:107
 [7] KnetArray at /kuacc/users/baristopal20/.julia/packages/Knet/C0PoK/src/knetarrays/karray.jl:75 [inlined]
 [8] similar at /kuacc/users/baristopal20/.julia/packages/Knet/C0PoK/src/knetarrays/abstractarray.jl:28 [inlined]
 [9] similar a

LoadError: Out of GPU memory trying to allocate 200.000 MiB
Effective GPU memory usage: 99.67% (14.707 GiB/14.756 GiB)
CUDA allocator usage: 14.054 GiB
binned usage: 4.047 KiB (4.047 KiB allocated, 0 bytes cached)
Discrepancy of 14.054 GiB between memory pool and allocator!


In [5]:
import Pkg

Pkg.status("PyCall")

No Matches in `/scratch/users/baristopal20/.julia/environments/v1.5/Project.toml`
